In [60]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import numpy as np

In [38]:
d = pd.read_csv('diabetes.csv')
d.shape

(768, 9)

In [25]:
d.Outcome.value_counts()
print(500/268, "Unbalanced") 

1.8656716417910448 Unbalanced


In [9]:
X = d.drop('Outcome', axis='columns')
y = d['Outcome']

In [13]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled[:3]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415]])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)

In [63]:
lr = cross_val_score(DecisionTreeClassifier(), X_train, y_train, cv=5)
np.average(lr)

0.7732889029937586

In [72]:
bg = BaggingClassifier(
    base_estimator=SVC(),
    n_estimators=100, # try and error
    max_samples=0.8, #porcentage of samples
    oob_score=True, #oob=out of bag
    random_state=0
)

In [36]:
bg.fit(X_train, y_train)
bg.oob_score_

0.7534722222222222

In [39]:
bg.score(X_test, y_test)

0.7760416666666666

In [42]:
av = cross_val_score(bg, X, y, cv=5)
np.average(av)

0.7578728461081402

In [44]:
mn = cross_val_score(RandomForestClassifier(), X, y, cv=5)
np.average(mn)

0.7682709447415329

In [77]:
df = pd.read_csv('heart.csv')
df.head(2)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1


In [78]:
df[df.Cholesterol>(df.Cholesterol.mean()+3*df.Cholesterol.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
76,32,M,ASY,118,529,0,Normal,130,N,0.0,Flat,1
149,54,M,ASY,130,603,1,Normal,125,Y,1.0,Flat,1
616,67,F,NAP,115,564,0,LVH,160,N,1.6,Flat,0


In [80]:
df1 = df[df.Cholesterol<=(df.Cholesterol.mean()+3*df.Cholesterol.std())]

In [82]:
df2 = df1[df1.Oldpeak<=(df1.Oldpeak.mean()+3*df1.Oldpeak.std())]
df2.shape

(909, 12)

In [84]:
df3 = df2[df2.RestingBP<=(df2.RestingBP.mean()+3*df2.RestingBP.std())]
df3.shape

(902, 12)

In [85]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

hle = df3
to_encode = ['Sex', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
for x in to_encode:
    hle[x] = le.fit_transform(hle[x])
hle = pd.get_dummies(hle, drop_first=True)
hle.head()

<ipython-input-85-8f36620c14ba>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hle[x] = le.fit_transform(hle[x])


,Age,Sex,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,1,140,289,0,1,172,0,0.0,2,0,1,0,0
1,49,0,160,180,0,1,156,0,1.0,1,1,0,1,0
2,37,1,130,283,0,2,98,0,0.0,2,0,1,0,0
3,48,0,138,214,0,1,108,1,1.5,1,1,0,0,0
4,54,1,150,195,0,1,122,0,0.0,2,0,0,1,0


In [86]:
x = hle.drop(['HeartDisease'], axis='columns')
y = hle['HeartDisease']

In [87]:
X_scaled = scaler.fit_transform(x)
X_scaled[:3]

array([[-1.42896269,  0.51485643,  0.46089071,  0.85238015, -0.55087949,
         0.01233655,  1.38191324, -0.82065181, -0.84676261,  1.0456339 ,
         2.06757196, -0.53547478, -0.22914788],
       [-0.47545956, -1.94228905,  1.5925728 , -0.16132855, -0.55087949,
         0.01233655,  0.75291107, -0.82065181,  0.14079864, -0.62072967,
        -0.4836591 ,  1.86750159, -0.22914788],
       [-1.74679706,  0.51485643, -0.10495034,  0.79657967, -0.55087949,
         1.60198908, -1.5272218 , -0.82065181, -0.84676261,  1.0456339 ,
         2.06757196, -0.53547478, -0.22914788]])

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)

In [89]:
sv = cross_val_score(SVC(), X_train, y_train, cv=5)
np.average(sv)

0.8505664488017428

In [90]:
bg.fit(X_train, y_train)
bg.oob_score_

0.8550295857988166

In [91]:
bg.score(X_test, y_test)

0.8893805309734514